In [1]:
"""
       Created on Jun 7 2018
       @author: Eric Lam
       @Collaborated histogram w/ Conor Morrison
"""
import numpy as np
from scipy import stats
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.metrics import *
plt.clf()

In [2]:
DATA_ROOT = '../'
CSV_FILE_PATH = os.path.join('cdr_from_2018-05-01-order-calldate-asc.csv')


In [3]:
HEADER = ["ID","calldate","callend","duration","connect_duration","progress_time","first_rtp_time","caller",
          "caller_domain","caller_reverse","callername","callername_reverse","called","called_domain","called_reverse",
          "sipcallerip","sipcallerport","sipcalledip","sipcalledport","whohanged","bye","lastSIPresponse_id",
          "lastSIPresponseNum","sighup","dscp","a_index","b_index","a_payload","b_payload","a_saddr","b_saddr",
          "a_received","b_received","a_lost","b_lost","a_ua_id","b_ua_id","a_avgjitter_mult10","b_avgjitter_mult10",
          "a_maxjitter","b_maxjitter","a_sl1","a_sl2","a_sl3","a_sl4","a_sl5","a_sl6","a_sl7","a_sl8","a_sl9","a_sl10",
          "a_d50","a_d70","a_d90","a_d120","a_d150","a_d200","a_d300","b_sl1","b_sl2","b_sl3","b_sl4","b_sl5","b_sl6","b_sl7",
          "b_sl8","b_sl9","b_sl10","b_d50","b_d70","b_d90","b_d120","b_d150","b_d200","b_d300","a_mos_lqo_mult10",
          "b_mos_lqo_mult10","a_mos_f1_mult10","a_mos_f2_mult10","a_mos_adapt_mult10","b_mos_f1_mult10","b_mos_f2_mult10",
          "b_mos_adapt_mult10","a_rtcp_loss","a_rtcp_maxfr","a_rtcp_avgfr_mult10","a_rtcp_maxjitter","a_rtcp_avgjitter_mult10",
          "b_rtcp_loss","b_rtcp_maxfr","b_rtcp_avgfr_mult10","b_rtcp_maxjitter","b_rtcp_avgjitter_mult10","a_last_rtp_from_end",
          "b_last_rtp_from_end","payload","jitter_mult10","mos_min_mult10","a_mos_min_mult10","b_mos_min_mult10",
          "packet_loss_perc_mult1000","a_packet_loss_perc_mult1000","b_packet_loss_perc_mult1000","delay_sum","a_delay_sum",
          "b_delay_sum","delay_avg_mult100","a_delay_avg_mult100","b_delay_avg_mult100","delay_cnt","a_delay_cnt",
          "b_delay_cnt","rtcp_avgfr_mult10","rtcp_avgjitter_mult10","lost","id_sensor","price_operator_mult100",
          "price_operator_currency_id","price_customer_mult100","price_customer_currency_id","reason_sip_cause",
          "reason_sip_text_id","reason_q850_cause","reason_q850_text_id","caller_silence","called_silence",
          "caller_silence_end","called_silence_end","a_mos_xr_min_mult10","b_mos_xr_min_mult10","a_mos_xr_mult10",
          "b_mos_xr_mult10","response_time_100","response_time_xxx","a_mos_f1_min_mult10","a_mos_f2_min_mult10",
          "a_mos_adapt_min_mult10","b_mos_f1_min_mult10","b_mos_f2_min_mult10","b_mos_adapt_min_mult10","a_rtp_ptime",
          "b_rtp_ptime","flags"]

In [ ]:
df = pd.read_csv(CSV_FILE_PATH, names=HEADER)


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (95,97,98,99,132) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df = df[HEADER]
#df['duration_td'] = df.callend - df.calldate

In [ ]:
#### procure data -> Delay_Sum, Jitter, MOS
sum_pkt_loss_a =  df['a_sl1'] + df['a_sl2'] + df['a_sl3'] +\
      df['a_sl4'] + df['a_sl5'] + df['a_sl6'] + df['a_sl7'] + \
      df['a_sl8'] + df['a_sl9'] + df['a_sl10'] 

sum_pkt_loss_b =  df['b_sl1'] + df['b_sl2'] + df['b_sl3'] +\
      df['b_sl4'] + df['b_sl5'] + df['b_sl6'] + df['b_sl7'] + \
      df['b_sl8'] + df['b_sl9'] + df['b_sl10'] 
# avg_pkt_loss = optional b/c 0's can be in either row for the same thing

In [ ]:

xi_a = df[['delay_sum','a_maxjitter','a_mos_adapt_mult10','a_mos_f1_mult10', 'a_mos_f2_mult10']].values
x_a = np.zeros((xi_a.shape[0], xi_a.shape[1]+1))
x_a[:,:-1] = xi_a
x_a[:,-1] = np.ones(xi_a.shape[0])

xi_b = df[['delay_sum','b_maxjitter','b_mos_adapt_mult10','b_mos_f1_mult10', 'b_mos_f2_mult10']].values
x_b = np.zeros((xi_b.shape[0], xi_b.shape[1]+1))
x_b[:,:-1] = xi_b
x_b[:,-1] = np.ones(xi_b.shape[0])

In [ ]:
####### Multi-Linear Regression W/ Packet Loss (A) & MOS

fig1 = plt.figure(1)
y = sum_pkt_loss_a.values
m = np.linalg.lstsq(x_a,y)
yfit = x_a.dot(m[0])
# print (yfit.shape, y.shape, m[0].shape, x_a.shape)
i = range(len(y))
print(m[0])
plt.plot(i,y,'ro')
plt.plot(i,yfit,':')
plt.title('Multi-Linear Regression w/ Packet Loss & MOS Version (A)')
plt.xlabel('Data Points Numbers')
plt.ylabel('Regression')
fig1.show()
# print('x_a ',x_a,'\n y \n', y)
reg_score = np.corrcoef(y, yfit)[0, 1]**2
print('Regression Score of Packet Loss & Mos Version A', reg_score)

# plt.show()


In [ ]:
####### Multi-Linear Regression W/ Packet Loss (B) & MOS
fig2 = plt.figure(2)
y0 = sum_pkt_loss_b.values
m0 = np.linalg.lstsq(x_b,y0)
yfit0 = x_b.dot(m0[0])
# print (yfit0.shape, y0.shape, m0[0].shape, x_b.shape)
i0 = range(len(y0))
print(m0[0])
plt.plot(i, y0, 'ro')
plt.plot(i,yfit0,':')
plt.title('Multi-Linear Regression w/ Packet Loss & MOS Version (B)')
plt.xlabel('Data Points Numbers')
plt.ylabel('Regression')
fig2.show()
reg_score0 = np.corrcoef(y0, yfit0)[0, 1]**2
print('Regression Score of Packet Loss & Mos Version B', reg_score0)
# plt.show()

#######################################################################


In [ ]:
'''
Multi-Linear regression: data is extremely noisy and messy
- no relationship between the variables 
- correlation score is extremely poor -> reject correlation between the variables 
- indication poor 

'''

In [ ]:
mindurcalls=df
mindurcalls['connect_duration'].fillna(value=0, inplace=True)
mindurcalls = mindurcalls[~pd.isnull(mindurcalls['connect_duration'])]
binwidth = 5
bins_hist = range(0, 45 + binwidth, binwidth)

###### Histogram part A
fig3 = plt.figure(3)

mos_a1 = mindurcalls['a_mos_f1_mult10'].fillna(0)
mos_a2 = mindurcalls['a_mos_f2_mult10'].fillna(0)
mos_adapt_a = mindurcalls['a_mos_adapt_mult10'].fillna(0)

hist_dataA = [mos_a1, mos_a2, mos_adapt_a]
hist_labels = ['a_mos_f1_mult10','a_mos_f2_mult10','a_mos_adapt_mult10']
plt.hist(hist_dataA, bins = bins_hist , alpha=1, label= hist_labels)
plt.legend(loc='upper left')
plt.grid(True)
plt.axis([0, 45, 0, 6000],'scaled')
plt.xlabel('MOS Scores')
plt.ylabel('Frequency')
plt.title('Histogram w/ MOS (A)')
fig3.show()

In [ ]:
###### Histogram part B
fig4 = plt.figure(4)

mos_b1 = mindurcalls['b_mos_f1_mult10'].fillna(0)
mos_b2 = mindurcalls['b_mos_f2_mult10'].fillna(0)
mos_adapt_b = mindurcalls['b_mos_adapt_mult10'].fillna(0)

hist_dataB = [mos_b1, mos_b2, mos_adapt_b]
hist_labels = ['b_mos_f1_mult10','b_mos_f2_mult10','b_mos_adapt_mult10']
plt.hist(hist_dataB, bins_hist, alpha=1, label= hist_labels)
plt.legend(loc='upper left')
plt.grid(True)
plt.axis([0, 45, 0, 6000],'scaled')
plt.xlabel('MOS Scores')
plt.ylabel('Frequency')
plt.title('Histogram w/ MOS (B)')
fig4.show()

plt.show()

In [ ]:
'''
Histogram
'''